<h1 align="center">Introduction to Machine Learning - 25737-2</h1>
<h4 align="center">Dr. R. Amiri</h4>
<h4 align="center">Sharif University of Technology, Spring 2024</h4>


<h4 align="center">Prject Phase-1</h4>


<h4 align="center">Seyed Mohammad Ojagh Kazazi (400100554) </h4>
<h4 align="center">Mohammad Morsali  (400101956) </h4>



Lybraries:

In [ ]:
import os
import shutil
from PIL import Image
import zipfile
import time
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, TensorDataset, ConcatDataset
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np
from google.colab import files
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
!pip install -U -q PyDrive
!pip install -U -q pydrive2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.7 MB/s eta 0:00:00


# **Simulation Question 1.**

Implement the learning phase of the algorithm, and select the based
models to be pretrained ResNet 18(you will need to add some fully connected layer(s) in
order to make it suitable for the given dataset!). Use S = 5, 10, 20 and R = 5, 10, 20 and train
different models on your data in each case, and report F1-score, accuracy, precision, recall and
AUROC of all of your models with all of your proposed aggregation methods.


In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Implement the ResNet18 model with additional fully connected layers for CIFAR-10
class ModifiedResNet18(nn.Module):
    def __init__(self):
        super(ModifiedResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        return self.resnet(x)

In [ ]:
# Authenticate and initialize PyDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:


# Define SISA algorithm components
class SISA:
    def __init__(self, S, R, epochs):
        self.S = S
        self.R = R
        self.epochs = epochs
        self.models = []  # Final models for each shard
        self.slice_models = {}  # Dictionary to save models after each slice


        # Variables to track training and unlearning times
        self.training_time = 0
        self.unlearning_time = 0

        # Create a directory to save models
        self.model_dir = f'models_S{self.S}_R{self.R}'
        if os.path.exists(self.model_dir):
            shutil.rmtree(self.model_dir)
        os.makedirs(self.model_dir)

    def shard_data(self, data):
        # Shard the data into S subsets
        shard_size = len(data) // self.S
        shards = [Subset(data, list(range(i * shard_size, (i + 1) * shard_size))) for i in range(self.S)]
        return shards

    def slice_data(self, shard):
        # Slice each shard into R slices
        slice_size = len(shard) // self.R
        slices = [Subset(shard, list(range(i * slice_size, (i + 1) * slice_size))) for i in range(self.R)]
        return slices

    def train(self, data):
        start_time = time.time()  # Start timing
        k = 0
        shards = self.shard_data(data)
        idx_shard = 0
        for shard in shards:
            idx_shard += 1
            slices = self.slice_data(shard)
            model = ModifiedResNet18().to(device)
            optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
            criterion = nn.CrossEntropyLoss()
            idx_slice = 0
            for slice in slices:
                idx_slice += 1
                train_loader = torch.utils.data.DataLoader(slice, batch_size=64, shuffle=True)
                for epoch in range(self.epochs):
                    for inputs, labels in train_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        #print(k)
                        k += 64
                        optimizer.zero_grad()
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()
                    print("@ shard:", idx_shard, " @ slice:", idx_slice, "@ epoch:", epoch, "  loss=", loss.item())
                # Save the model after training each slice
                model_path = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice}.pth')
                torch.save(model.state_dict(), model_path)
                self.slice_models[(idx_shard, idx_slice)] = model_path
            self.models.append(model)
        self.training_time = time.time() - start_time  # Record training time





    def load_models(self):
        self.models = []
        self.slice_models = {}
        for idx_shard in range(1, self.S + 1):
            for idx_slice in range(1, self.R + 1):
                model = ModifiedResNet18().to(device)
                model_path = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice}.pth')
                self.slice_models[(idx_shard, idx_slice)] = model_path
                if idx_slice == self.R:
                  print("loading shard:",idx_shard)
                  model.load_state_dict(torch.load(model_path))
                  self.models.append(model)


    def zip_models(self):
        zip_filename = f'{self.model_dir}.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for root, dirs, files in os.walk(self.model_dir):
                for file in files:
                    zipf.write(os.path.join(root, file), arcname=file)
        return zip_filename

    def upload_models_to_drive(self):
        zip_filename = self.zip_models()
        file = drive.CreateFile({'title': os.path.basename(zip_filename)})
        file.SetContentFile(zip_filename)
        file.Upload()
        print(f'Uploaded file with ID {file.get("id")}')
        return file.get("id")

    def download_and_extract_models_from_drive(self, file_id):
        file = drive.CreateFile({'id': file_id})
        zip_filename = f'{self.model_dir}.zip'
        file.GetContentFile(zip_filename)
        print(f"Downloaded {zip_filename} from Google Drive")
        self.extract_and_load_model(zip_filename)


    def extract_and_load_model(self, zip_filename):
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(self.model_dir)
        self.load_models()






    def predict_for_all_models(self, test_loader):
        #print()
        #print("in predict_for_all_models:")
        #print("test_loader:")
        #print(test_loader)
        all_predictions = []
        for model in self.models:
            model.eval()  # Set model to evaluation mode
            model_predictions = []
            for inputs, _ in test_loader:
                #print("input:",input)
                inputs = inputs.to(device)
                # Get the predictions from the current model
                with torch.no_grad():
                    pred = model(inputs)
                    #print(pred)
                model_predictions.append(pred.cpu())
            all_predictions.append(torch.cat(model_predictions))
        #print("all_predictions:")
        #print(all_predictions)
        #print("_____")
        return all_predictions

    def aggregate_majority_vote(self, test_loader):
        all_predictions = self.predict_for_all_models(test_loader)
        #print("aggregate_majority_vote")
        #print(all_predictions)
        # Aggregate the predictions using a majority vote
        stacked_predictions = torch.stack([torch.argmax(pred, dim=1) for pred in all_predictions])
        aggregated_predictions = torch.mode(stacked_predictions, dim=0)[0]

        prob_array = torch.zeros(10, stacked_predictions.shape[1])
        #print(all_predictions)
        #print(stacked_predictions)

        for col in range(stacked_predictions.shape[1]):
            for num in range(10):
                prob_array[num, col] = torch.mean((stacked_predictions[:, col] == num).float())

        #print(prob_array)

        return aggregated_predictions, prob_array.T

    def aggregate_weighted_average(self, test_loader, validation_loader):
        all_predictions = self.predict_for_all_models(test_loader)
        model_accuracies = self.compute_model_accuracies(validation_loader)
        weighted_predictions = torch.zeros_like(all_predictions[0])
        for model_pred, accuracy in zip(all_predictions, model_accuracies):
            weighted_predictions += model_pred * accuracy
        weighted_predictions = F.softmax(weighted_predictions, dim=1)
        aggregated_predictions = torch.argmax(weighted_predictions, dim=1)
        #print(weighted_predictions)
        return aggregated_predictions, weighted_predictions

    def compute_model_accuracies(self, validation_loader):
        accuracies = []
        for model in self.models:
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in validation_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            accuracy = correct / total
            accuracies.append(accuracy)
        return torch.tensor(accuracies)

    def aggregate_simple_average(self, test_loader):
        all_predictions = self.predict_for_all_models(test_loader)
        # Aggregate the predictions by simple averaging of probabilities
        averaged_probabilities = torch.mean(torch.stack(all_predictions), dim=0)
        aggregated_predictions = torch.argmax(averaged_probabilities, dim=1)
        #print(averaged_probabilities)
        averaged_probabilities = F.softmax(averaged_probabilities, dim=1)
        return aggregated_predictions, averaged_probabilities

    def evaluate(self, test_loader, aggregation_function):
        aggregated_predictions, all_probs = aggregation_function(test_loader)
        #print(aggregated_predictions)
        all_labels = []
        with torch.no_grad():
            for _, labels in test_loader:
                all_labels.extend(labels.numpy())
        all_preds = aggregated_predictions.cpu().numpy().tolist()
        all_probs = all_probs.cpu().numpy().tolist()
        all_labels = np.array(all_labels).tolist()
        print("all_preds=", all_preds)
        print("all_labels", all_labels)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        auroc = roc_auc_score(all_labels, all_probs, multi_class='ovr')

        return accuracy, f1, precision, recall, auroc

    def evaluate_with_weighted_aggregate(self, test_loader, aggregation_function, validation_loader):
        aggregated_predictions, all_probs = aggregation_function(test_loader, validation_loader)
        all_labels = []
        with torch.no_grad():
            for _, labels in test_loader:
                all_labels.extend(labels.numpy())
        all_preds = aggregated_predictions.cpu().numpy().tolist()
        all_probs = all_probs.cpu().numpy().tolist()
        all_labels = np.array(all_labels).tolist()
        print("all_preds=", all_preds)
        print("all_labels", all_labels)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        auroc = roc_auc_score(all_labels, all_probs, multi_class='ovr')

        return accuracy, f1, precision, recall, auroc

    # Unlearning Algorithm
    def unlearn(self, dataset, num_samples=500):
        start_time = time.time()  # Start timing
        # Randomly choose samples to forget
        forget_indices = np.random.choice(len(dataset), num_samples, replace=False)
        print(forget_indices)


        # Reinitialize models only for affected slices
        shards = self.shard_data(dataset)
        for idx_shard, shard in enumerate(shards, start=1):
            slices = self.slice_data(shard)
            flag = False
            for idx_slice, slice in enumerate(slices, start=1):
                slice_start_idx = (idx_shard - 1) * len(dataset) // self.S
                slice_end_idx = slice_start_idx + idx_slice * len(shard) // self.R
                slice_indices = set(range(slice_start_idx, slice_end_idx))

                print(slice_start_idx," ---> ",slice_end_idx)
                overlap = slice_indices.intersection(forget_indices)
                if overlap:
                    difference = {idx - slice_start_idx for idx in overlap}
                    print("overlap: ", overlap)
                    print("difference: ", difference)
                    flag = True
                    model = ModifiedResNet18().to(device)
                    if idx_slice != 1:  # Avoid loading for the first slice
                        model_path_prepart = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice - 1}.pth')
                        model.load_state_dict(torch.load(model_path_prepart))
                    # Create new slice data without the forgotten indices
                    retained_indices = list(set(range(len(slice))) - difference)
                    retained_set = Subset(slice, retained_indices)
                    retained_loader = DataLoader(retained_set, batch_size=64, shuffle=True)
                    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
                    criterion = nn.CrossEntropyLoss()
                    model.train()
                    for epoch in range(self.epochs):
                        for inputs, labels in retained_loader:
                            inputs, labels = inputs.to(device), labels.to(device)
                            optimizer.zero_grad()
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                            loss.backward()
                            optimizer.step()
                        print(f"@ shard: {idx_shard}, @ slice: {idx_slice}, @ epoch: {epoch + 1}, loss: {loss.item()}")

                    # Save the updated model
                    model_path = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice}.pth')
                    if os.path.exists(model_path):
                        os.remove(model_path)
                    torch.save(model.state_dict(), model_path)
                    self.slice_models[(idx_shard, idx_slice)] = model_path

            if flag == False:
                  print("shard:",idx_shard," model dont change")

        self.unlearning_time = time.time() - start_time  # Record unlearning time


    # Unlearning Algorithm
    def unlearn_with_indices(self, dataset, forget_indices):
        start_time = time.time()  # Start timing
        print(forget_indices)


        # Reinitialize models only for affected slices
        shards = self.shard_data(dataset)
        for idx_shard, shard in enumerate(shards, start=1):
            slices = self.slice_data(shard)
            flag = False
            for idx_slice, slice in enumerate(slices, start=1):
                slice_start_idx = (idx_shard - 1) * len(dataset) // self.S
                slice_end_idx = slice_start_idx + idx_slice * len(shard) // self.R
                slice_indices = set(range(slice_start_idx, slice_end_idx))

                print(slice_start_idx," ---> ",slice_end_idx)
                overlap = slice_indices.intersection(forget_indices)
                if overlap:
                    difference = {idx - slice_start_idx for idx in overlap}
                    print("overlap: ", overlap)
                    print("difference: ", difference)
                    flag = True
                    model = ModifiedResNet18().to(device)
                    if idx_slice != 1:  # Avoid loading for the first slice
                        print("idx_slice != 1")
                        model_path_prepart = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice - 1}.pth')
                        model.load_state_dict(torch.load(model_path_prepart))
                    # Create new slice data without the forgotten indices
                    retained_indices = list(set(range(len(slice))) - difference)
                    retained_set = Subset(slice, retained_indices)
                    retained_loader = DataLoader(retained_set, batch_size=64, shuffle=True)
                    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
                    criterion = nn.CrossEntropyLoss()
                    model.train()
                    for epoch in range(self.epochs):
                        for inputs, labels in retained_loader:
                            inputs, labels = inputs.to(device), labels.to(device)
                            optimizer.zero_grad()
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                            loss.backward()
                            optimizer.step()
                        print(f"@ shard: {idx_shard}, @ slice: {idx_slice}, @ epoch: {epoch + 1}, loss: {loss.item()}")

                    # Save the updated model
                    model_path = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice}.pth')
                    if os.path.exists(model_path):
                        os.remove(model_path)
                    torch.save(model.state_dict(), model_path)
                    self.slice_models[(idx_shard, idx_slice)] = model_path

            if flag == False:
                  print("shard:",idx_shard," model dont change")

        self.unlearning_time = time.time() - start_time  # Record unlearning time


**Trainig Data:**

In [ ]:
# transform to include resizing to 224x224
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:12<00:00, 13327880.22it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


## **S=5 R=5 :**

**Initialize SISA**

In [ ]:
S = 5
R = 5
epochs = 5

sisa_S5_R5 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1YQwzzpRoQuNQ1ubKp5LDbthBdSnMVIoP'

# Download and extract models
sisa_S5_R5.download_and_extract_models_from_drive(file_id)

Downloaded models_S5_R5.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:02<00:00, 23.0MB/s]


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


In [ ]:
sisa_S5_R5.extract_and_load_model('/content/models_S5_R5.zip')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


In [ ]:
# Load models
sisa_S5_R5.load_models()

# Verify loaded models
print(f"Loaded {len(sisa_S5_R5.models)} models.")

**Training:**

In [ ]:
sisa_S5_R5.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 65.8MB/s]


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 1.0347992181777954
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 0.7653300762176514
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 0.29437288641929626
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 0.10116563737392426
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.12522274255752563
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 0.5618718266487122
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.09842759370803833
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.32962387800216675
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.047447845339775085
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.0023070734459906816
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.5936269164085388
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.6736375093460083
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.09980056434869766
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.08890807628631592
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.02499321661889553
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.592962384223938
@ shard: 1  

Upload model on google drive:

In [ ]:
sisa_S5_R5.upload_models_to_drive()

Uploaded file with ID 1YQwzzpRoQuNQ1ubKp5LDbthBdSnMVIoP


'1YQwzzpRoQuNQ1ubKp5LDbthBdSnMVIoP'

Test Data:

In [ ]:
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Get a sample of 100 test examples
sample_indices = torch.randperm(len(cifar10_test))[:1000]
test_data = torch.utils.data.Subset(cifar10_test, sample_indices)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:
# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate_with_weighted_aggregate(test_loader, sisa_S5_R5.aggregate_weighted_average, validation_loader)

all_preds= [8, 6, 0, 4, 1, 1, 7, 5, 2, 9, 3, 1, 8, 0, 4, 6, 2, 3, 2, 8, 6, 4, 9, 2, 4, 3, 0, 9, 9, 9, 1, 6, 8, 2, 5, 1, 5, 2, 6, 6, 9, 7, 1, 7, 7, 3, 7, 3, 5, 7, 4, 8, 9, 5, 0, 3, 7, 4, 2, 7, 8, 5, 7, 3, 7, 1, 5, 1, 8, 8, 0, 6, 7, 4, 3, 4, 3, 0, 0, 7, 2, 0, 2, 4, 9, 8, 0, 6, 0, 5, 4, 1, 0, 4, 2, 3, 2, 9, 0, 9, 0, 4, 1, 7, 2, 7, 3, 9, 4, 8, 6, 0, 7, 9, 9, 8, 1, 4, 6, 5, 2, 3, 5, 8, 9, 8, 0, 9, 0, 4, 1, 2, 9, 6, 0, 7, 5, 8, 1, 1, 3, 1, 3, 1, 4, 4, 4, 8, 6, 1, 5, 5, 2, 3, 9, 4, 8, 9, 4, 2, 8, 6, 0, 0, 1, 8, 8, 5, 3, 6, 2, 7, 0, 8, 5, 9, 0, 7, 0, 6, 1, 6, 8, 2, 6, 0, 7, 5, 0, 8, 3, 2, 5, 3, 6, 7, 0, 0, 8, 4, 4, 4, 4, 2, 2, 0, 6, 4, 2, 3, 8, 7, 4, 0, 0, 8, 4, 1, 4, 8, 7, 2, 5, 8, 6, 7, 1, 0, 5, 9, 4, 4, 3, 3, 2, 3, 2, 0, 5, 0, 7, 2, 4, 6, 4, 0, 5, 5, 1, 6, 4, 5, 7, 5, 9, 4, 4, 1, 7, 0, 0, 1, 7, 3, 7, 8, 4, 4, 4, 7, 6, 2, 8, 1, 0, 1, 8, 9, 1, 5, 9, 3, 4, 6, 0, 8, 6, 6, 2, 5, 4, 6, 0, 1, 0, 6, 1, 1, 7, 2, 5, 2, 1, 5, 1, 6, 3, 7, 3, 5, 5, 3, 0, 6, 0, 8, 4, 1, 0, 9, 1, 8, 9, 8, 6, 9, 7, 1, 7, 9

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9230, F1-score: 0.9228, Precision: 0.9235, Recall: 0.9230, AUROC: 0.9951


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate(test_loader, sisa_S5_R5.aggregate_simple_average)

all_preds= [5, 7, 9, 4, 3, 3, 7, 0, 8, 5, 4, 4, 3, 3, 6, 7, 4, 3, 5, 9, 0, 4, 8, 0, 0, 1, 9, 2, 2, 3, 3, 8, 7, 3, 2, 3, 2, 2, 7, 7, 7, 9, 5, 5, 9, 1, 1, 9, 4, 7, 9, 4, 0, 2, 8, 3, 0, 6, 0, 6, 9, 7, 8, 6, 2, 7, 2, 8, 7, 3, 4, 8, 7, 2, 8, 5, 6, 9, 8, 8, 6, 4, 8, 1, 9, 0, 0, 7, 2, 9, 5, 8, 2, 8, 4, 2, 4, 8, 6, 7, 4, 3, 3, 4, 9, 7, 5, 9, 1, 6, 0, 5, 5, 1, 2, 7, 4, 2, 3, 4, 6, 8, 2, 7, 5, 6, 5, 7, 8, 7, 2, 6, 6, 7, 0, 7, 4, 8, 4, 0, 6, 3, 7, 5, 4, 1, 8, 9, 7, 2, 3, 4, 8, 4, 1, 0, 0, 9, 1, 1, 2, 9, 4, 8, 1, 9, 8, 3, 2, 9, 1, 4, 4, 4, 5, 3, 8, 6, 6, 7, 2, 4, 3, 9, 2, 8, 9, 1, 6, 0, 5, 5, 0, 4, 0, 4, 4, 9, 8, 1, 4, 1, 1, 6, 2, 5, 0, 4, 7, 0, 2, 8, 4, 7, 8, 8, 9, 9, 0, 3, 5, 9, 6, 1, 1, 9, 2, 9, 4, 1, 7, 7, 5, 5, 0, 0, 7, 7, 3, 8, 6, 3, 8, 2, 6, 1, 9, 1, 9, 4, 4, 3, 1, 5, 4, 7, 9, 0, 6, 7, 0, 6, 5, 0, 6, 2, 5, 1, 1, 1, 7, 9, 8, 4, 1, 9, 5, 2, 3, 2, 2, 7, 2, 1, 9, 9, 3, 4, 7, 1, 5, 9, 6, 6, 5, 9, 7, 9, 1, 2, 7, 5, 2, 7, 9, 8, 4, 8, 3, 9, 4, 2, 6, 8, 2, 7, 1, 3, 5, 5, 9, 3, 8, 2, 2, 0, 5, 3, 7, 7

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9270, F1-score: 0.9269, Precision: 0.9278, Recall: 0.9270, AUROC: 0.9957


*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate(test_loader, sisa_S5_R5.aggregate_majority_vote)

all_preds= [5, 7, 9, 4, 3, 3, 7, 0, 8, 5, 4, 4, 2, 3, 6, 7, 4, 3, 5, 0, 4, 4, 8, 0, 0, 1, 9, 2, 2, 3, 3, 8, 7, 3, 2, 3, 2, 2, 7, 7, 7, 9, 5, 5, 9, 1, 1, 9, 4, 3, 9, 4, 0, 2, 8, 3, 0, 6, 0, 6, 9, 7, 8, 6, 2, 7, 2, 8, 7, 3, 4, 8, 7, 2, 8, 5, 6, 9, 8, 8, 6, 4, 8, 1, 9, 0, 0, 7, 2, 9, 5, 8, 2, 8, 4, 2, 4, 8, 6, 7, 4, 5, 3, 4, 9, 4, 5, 9, 1, 6, 0, 5, 5, 1, 2, 7, 4, 2, 3, 4, 6, 8, 2, 7, 5, 6, 5, 7, 8, 7, 2, 6, 6, 7, 2, 7, 4, 8, 4, 0, 6, 3, 7, 5, 4, 1, 8, 9, 5, 2, 3, 4, 8, 4, 1, 0, 0, 9, 1, 1, 2, 9, 4, 8, 1, 9, 8, 3, 2, 9, 1, 4, 4, 4, 5, 3, 8, 6, 6, 7, 2, 4, 3, 9, 2, 8, 9, 1, 6, 0, 5, 5, 0, 4, 0, 4, 4, 9, 8, 1, 4, 1, 1, 6, 2, 5, 0, 4, 7, 0, 2, 8, 4, 7, 8, 8, 9, 9, 0, 3, 0, 9, 6, 1, 1, 9, 2, 9, 4, 1, 7, 7, 5, 2, 0, 0, 7, 7, 3, 8, 6, 3, 8, 2, 6, 1, 9, 1, 9, 4, 4, 3, 1, 5, 4, 3, 9, 0, 6, 7, 0, 6, 5, 0, 6, 2, 5, 1, 1, 1, 7, 9, 8, 4, 1, 9, 5, 2, 3, 2, 2, 7, 2, 1, 9, 9, 3, 4, 7, 1, 5, 9, 6, 6, 5, 9, 7, 9, 1, 2, 7, 5, 2, 7, 9, 8, 4, 8, 3, 9, 4, 2, 6, 8, 2, 7, 1, 3, 5, 5, 9, 3, 8, 2, 2, 0, 5, 3, 7, 7

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9220, F1-score: 0.9220, Precision: 0.9233, Recall: 0.9220, AUROC: 0.9827


downloading:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir('/content')

In [ ]:
import shutil
from google.colab import files

# Create a zip archive of the folder
shutil.make_archive('model_shard1_slice1', 'zip', '/content/models_S5_R2/model_shard1_slice1.pth')

# Download the zip archive
files.download('model_shard1_slice1.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Simulation Question 2.**
Implement the unlearning algorithm on all your trained models in the previous question and ”forget” 500 randomly chosen data of the whole dataset and measure the performance using your proposed metric, also report F1-score, accuracy, precision, recall and AUROC of all of your models with all of your proposed aggregation methods.

## **S=5 R=5:**

In [ ]:
print(len(cifar10_train))

50000


In [ ]:
print(sisa_S5_R5.unlearning_time)

29.289684057235718


In [ ]:
sisa_S5_R5.unlearn(cifar10_train, 5)

[38022 19535 20514 30518 18589]
0  --->  2000
0  --->  4000
0  --->  6000
0  --->  8000
0  --->  10000
shard: 1  model dont change
10000  --->  12000
10000  --->  14000
10000  --->  16000
10000  --->  18000
10000  --->  20000
overlap:  {18589, 19535}
difference:  {8589, 9535}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 2, @ slice: 5, @ epoch: 1, loss: 0.7749964594841003
@ shard: 2, @ slice: 5, @ epoch: 2, loss: 0.32468366622924805
@ shard: 2, @ slice: 5, @ epoch: 3, loss: 0.022222964093089104
@ shard: 2, @ slice: 5, @ epoch: 4, loss: 0.009649545885622501
@ shard: 2, @ slice: 5, @ epoch: 5, loss: 0.28403493762016296
20000  --->  22000
overlap:  {20514}
difference:  {514}
@ shard: 3, @ slice: 1, @ epoch: 1, loss: 1.3384274244308472
@ shard: 3, @ slice: 1, @ epoch: 2, loss: 0.6571936011314392
@ shard: 3, @ slice: 1, @ epoch: 3, loss: 1.1527689695358276
@ shard: 3, @ slice: 1, @ epoch: 4, loss: 0.046080198138952255
@ shard: 3, @ slice: 1, @ epoch: 5, loss: 0.01761472411453724
20000  --->  24000
overlap:  {20514}
difference:  {514}
@ shard: 3, @ slice: 2, @ epoch: 1, loss: 0.5684215426445007
@ shard: 3, @ slice: 2, @ epoch: 2, loss: 0.05124233663082123
@ shard: 3, @ slice: 2, @ epoch: 3, loss: 0.18143363296985626
@ shard: 3, @ slice: 2, @ epoch: 4, loss: 0.019392868503928185
@ shard: 3, @ slice: 

*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate(test_loader, sisa_S5_R5.aggregate_majority_vote)

all_preds= [5, 7, 9, 4, 3, 3, 7, 0, 8, 5, 4, 4, 2, 3, 6, 7, 4, 3, 5, 0, 0, 4, 8, 0, 0, 1, 9, 2, 2, 3, 3, 8, 7, 3, 2, 3, 2, 2, 7, 7, 7, 9, 5, 5, 9, 1, 1, 9, 4, 7, 9, 4, 0, 2, 8, 3, 0, 6, 0, 6, 9, 7, 8, 6, 2, 7, 2, 8, 7, 3, 4, 8, 7, 2, 8, 5, 3, 9, 8, 8, 6, 4, 8, 1, 9, 0, 0, 7, 2, 9, 5, 8, 2, 8, 4, 2, 4, 8, 6, 7, 4, 3, 3, 4, 9, 7, 5, 9, 1, 6, 0, 5, 5, 1, 2, 7, 4, 2, 3, 4, 6, 8, 2, 7, 5, 6, 5, 7, 8, 7, 2, 6, 6, 7, 0, 7, 4, 8, 4, 0, 6, 3, 7, 5, 4, 1, 8, 9, 5, 2, 3, 4, 8, 4, 1, 0, 0, 9, 1, 1, 2, 9, 4, 8, 1, 9, 8, 3, 2, 9, 1, 4, 4, 4, 5, 3, 8, 6, 6, 7, 2, 4, 3, 9, 2, 8, 9, 1, 6, 0, 5, 5, 0, 4, 0, 4, 4, 9, 8, 1, 4, 1, 1, 6, 2, 5, 0, 4, 7, 0, 2, 8, 4, 7, 8, 8, 9, 9, 0, 3, 5, 9, 6, 1, 1, 9, 2, 9, 4, 1, 7, 7, 5, 2, 0, 0, 7, 7, 3, 8, 6, 3, 8, 2, 6, 1, 9, 1, 9, 4, 4, 3, 1, 5, 4, 3, 9, 0, 6, 7, 0, 6, 5, 0, 6, 2, 5, 1, 1, 1, 7, 9, 8, 4, 1, 9, 5, 2, 3, 2, 2, 7, 2, 1, 9, 9, 3, 4, 7, 1, 5, 9, 6, 6, 5, 9, 7, 9, 1, 2, 7, 5, 0, 7, 9, 8, 4, 8, 3, 9, 4, 2, 6, 8, 2, 7, 1, 3, 5, 5, 9, 3, 8, 2, 2, 0, 5, 3, 7, 7

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9210, F1-score: 0.9208, Precision: 0.9216, Recall: 0.9210, AUROC: 0.9833


In [ ]:
sisa_S5_R5.unlearn(cifar10_train, 50)

[36769 19683 22466 21230 21823 33539  3314 37978 15854 18208 39273 37453
  5926 41947 14920 10783 36018 33665 19284 45257  8427 18033 20316 39402
 45022 15410  4234 34028 29366 49500 31068 14688 34620  2591  9000 45645
  1225 16079 37586 12446 41216 10494 34355 31112 17919 30093  4937 22752
 27755 32779]
0  --->  2000
overlap:  {1225}
difference:  {1225}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 0.9304810762405396
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 0.8235521912574768
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 0.3339286744594574
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.2153344452381134
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.034059133380651474
0  --->  4000
overlap:  {1225, 3314, 2591}
difference:  {1225, 3314, 2591}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.6817817091941833
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.44427862763404846
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.25425413250923157
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.026945875957608223
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.02449473738670349
0  --->  6000
overlap:  {5926, 1225, 4234, 4937, 3314, 2591}
difference:  {5926, 4937, 1225, 4234, 3314, 2591}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 0.4855729639530182
@ shard: 1, @ slice: 3, @ epoch: 2, loss: 0.7400068640708923
@ shard: 1, @ slice: 3, @ epoch: 3, loss: 0.12359306961297989
@ shard: 1, @

# **Add On. Simulation Question 1.**
Randomly select 500 data points from a specific class in
the training set. For each selected data point, turn a 3x3 block of pixels to black, with the
position of the block chosen randomly. Train your best model of the previous questions, using
the poisoned dataset. Evaluate the model’s performance, using all the same metrics as before,
on clean test data to ensure general performance is not significantly degraded. Also, calculate
the attack success rate (ASR) as the percentage of test samples misclassified as the target class.

In [ ]:
# Step 1: Select 500 data points from a specific class in the training set
def select_and_poison_data_inplace(dataset, target_class, num_samples=500, block_size=3):
    class_indices = [i for i in range(len(dataset)) if dataset.targets[i] == target_class]
    selected_indices = random.sample(class_indices, num_samples)
    block_size = int(224 / 32) * block_size
    if block_size != 0:
      for idx in selected_indices:
          #print(dataset.targets[idx])
          img = dataset.data[idx]
          img = np.copy(img)  # To avoid modifying the original image
          # Randomly select a position for the 3x3 block
          x = random.randint(0, img.shape[0] - block_size)
          y = random.randint(0, img.shape[1] - block_size)
          img[x:x+block_size, y:y+block_size, :] = 0  # Turn the block to black
          dataset.data[idx] = img

    return selected_indices

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 43744949.76it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Step 2: Poison the data
poisoned_indices = select_and_poison_data_inplace(cifar10_train, target_class=0, num_samples=500, block_size=3)

In [ ]:
poisoned_indices = [42044, 3880, 33447, 8392, 36106, 31756, 29884, 22665, 34318, 20985, 3515, 45156, 43775, 30172, 1434, 43454, 555, 48032, 9362, 44599, 20790, 23808, 6867, 48908, 14592, 18951, 48856, 39083, 34998, 371, 8748, 35162, 21777, 21863, 40807, 15580, 13238, 10867, 43525, 47336, 26723, 9714, 6393, 47931, 23056, 4270, 49344, 13181, 47994, 17884, 18630, 29513, 33386, 37962, 30914, 29641, 18599, 41613, 41277, 1340, 1593, 41731, 49236, 48851, 30595, 25789, 36886, 47207, 36155, 47375, 16825, 30959, 34313, 48470, 8086, 11078, 21758, 39650, 27850, 22418, 42952, 29497, 12409, 902, 11100, 15755, 17620, 3076, 2932, 14393, 22026, 5477, 9384, 17133, 19814, 29854, 2718, 23096, 4798, 7674, 7889, 20523, 35626, 12192, 11709, 42688, 38519, 37978, 16838, 5703, 38224, 42231, 1711, 47379, 10578, 1188, 42998, 29341, 35411, 27931, 28449, 44299, 24096, 6563, 6276, 36606, 22997, 2391, 7251, 8313, 29880, 18315, 46706, 46200, 34547, 41975, 36819, 48119, 8116, 30257, 39664, 5623, 19011, 44946, 8515, 2066, 25070, 16238, 29367, 43831, 35248, 32460, 47274, 35286, 49004, 49091, 10553, 9874, 24735, 39486, 15827, 8660, 16501, 16370, 13467, 2950, 19041, 5770, 36558, 9257, 12328, 35179, 27211, 28223, 40652, 14509, 41243, 35490, 25531, 20452, 22189, 23732, 30116, 46486, 6833, 22317, 20205, 11356, 4165, 44956, 43944, 18211, 2287, 7991, 11734, 34168, 11835, 965, 17015, 48291, 37371, 14341, 43664, 49006, 11479, 41609, 13326, 39739, 8096, 2233, 16487, 15258, 11783, 439, 44746, 41234, 7504, 2453, 27533, 18803, 39883, 497, 31947, 32505, 36312, 48071, 13069, 33647, 28675, 557, 1335, 35326, 7409, 37747, 9123, 26427, 21903, 33453, 22674, 15086, 8549, 4991, 17303, 33325, 481, 13649, 38190, 42678, 13627, 29860, 7530, 7979, 47486, 28665, 29364, 30645, 26776, 44820, 12420, 46822, 33058, 33419, 38053, 44426, 1709, 32631, 42493, 20295, 11216, 48777, 26322, 18847, 45233, 3335, 23002, 38574, 5404, 35798, 39049, 17233, 37905, 40479, 7792, 16624, 28897, 16245, 11849, 48209, 32247, 34017, 44637, 19339, 49359, 9945, 4387, 15870, 34026, 3184, 1594, 199, 23863, 37999, 10955, 38029, 49308, 20267, 31837, 8155, 46873, 5371, 29518, 49557, 47068, 21639, 11374, 3689, 12807, 38977, 25350, 4061, 7423, 2189, 44294, 20686, 16137, 23507, 13860, 31955, 25391, 11838, 5115, 20584, 7727, 26076, 5322, 4097, 14275, 49656, 39584, 11775, 49856, 9759, 44017, 24676, 31506, 39477, 47916, 31896, 7925, 39709, 43002, 5640, 38083, 19106, 19282, 32629, 20070, 2848, 36789, 10061, 20539, 32479, 18192, 26045, 10365, 26743, 36693, 18106, 16548, 15396, 22858, 43092, 45314, 332, 7757, 115, 32056, 21564, 33421, 29039, 8217, 10650, 11014, 35927, 32033, 18603, 25746, 8102, 22326, 31127, 42110, 23245, 46935, 43731, 25178, 47906, 16866, 38718, 23629, 26599, 14528, 4141, 4229, 18605, 27780, 9221, 24802, 38091, 24567, 30141, 9751, 28113, 39012, 7347, 17221, 8444, 29295, 35685, 3466, 14727, 13841, 44031, 41164, 27175, 49174, 44035, 35820, 17517, 37240, 34101, 12507, 20710, 33797, 33530, 3362, 8553, 40193, 9767, 17963, 47966, 48109, 35417, 34748, 5956, 27502, 49992, 39631, 47579, 42361, 4048, 16952, 25816, 12325, 42934, 30744, 18854, 39288, 43403, 44719, 23822, 5549, 5673, 36531, 44715, 46086, 5580, 12165, 20257, 22680, 10548, 39037, 40321, 42446, 48546, 3066, 43303, 13613, 9438, 43829, 35846, 36955, 26184, 28794, 6479, 23891, 46619, 25942, 28185, 3246, 7196, 1980, 16554, 35725, 15707, 13008, 32140, 22724, 28092, 6354, 8941]

In [ ]:
print(poisoned_indices)

[42044, 3880, 33447, 8392, 36106, 31756, 29884, 22665, 34318, 20985, 3515, 45156, 43775, 30172, 1434, 43454, 555, 48032, 9362, 44599, 20790, 23808, 6867, 48908, 14592, 18951, 48856, 39083, 34998, 371, 8748, 35162, 21777, 21863, 40807, 15580, 13238, 10867, 43525, 47336, 26723, 9714, 6393, 47931, 23056, 4270, 49344, 13181, 47994, 17884, 18630, 29513, 33386, 37962, 30914, 29641, 18599, 41613, 41277, 1340, 1593, 41731, 49236, 48851, 30595, 25789, 36886, 47207, 36155, 47375, 16825, 30959, 34313, 48470, 8086, 11078, 21758, 39650, 27850, 22418, 42952, 29497, 12409, 902, 11100, 15755, 17620, 3076, 2932, 14393, 22026, 5477, 9384, 17133, 19814, 29854, 2718, 23096, 4798, 7674, 7889, 20523, 35626, 12192, 11709, 42688, 38519, 37978, 16838, 5703, 38224, 42231, 1711, 47379, 10578, 1188, 42998, 29341, 35411, 27931, 28449, 44299, 24096, 6563, 6276, 36606, 22997, 2391, 7251, 8313, 29880, 18315, 46706, 46200, 34547, 41975, 36819, 48119, 8116, 30257, 39664, 5623, 19011, 44946, 8515, 2066, 25070, 16238, 29

In [ ]:
# Initialize SISA
S = 5
R = 5
epochs = 5
poisone_sisa = SISA(S, R, epochs)

In [ ]:
poisone_sisa.extract_and_load_model('/content/models_S5_R5.zip')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


In [ ]:
# Train the model
poisone_sisa.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 178MB/s]


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 0.8654252886772156
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 0.3747510015964508
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 0.3422948718070984
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 0.24503670632839203
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.08936162292957306
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 0.4068167805671692
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.2681864798069
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.4361494183540344
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.0647049993276596
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.22512809932231903
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.4825003147125244
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.3507573902606964
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.28768378496170044
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.008365673944354057
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.012603973038494587
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.5619994401931763
@ shard: 1  @ slice

***UPLOADING:***

In [ ]:
poisone_sisa.upload_models_to_drive()

Uploaded file with ID 1HdZyTSCZiA8sycdg-RAb3oJi6XgDNCcp


'1HdZyTSCZiA8sycdg-RAb3oJi6XgDNCcp'

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1HdZyTSCZiA8sycdg-RAb3oJi6XgDNCcp'

# Download and extract models
poisone_sisa.download_and_extract_models_from_drive(file_id)

Downloaded models_S5_R5.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


In [ ]:
# Step 4: Evaluate the model's performance on clean test data
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [ ]:
# Get a sample of 1000 test examples from class 3
class_3_indices = [i for i, (_, label) in enumerate(cifar10_test) if label == 0]
sample_indices = torch.randperm(len(class_3_indices))[:1000]
sampled_class_3_indices = [class_3_indices[i] for i in sample_indices]
test_data = torch.utils.data.Subset(cifar10_test, class_3_indices)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

In [ ]:
# Get a sample of 100 test examples
sample_indices = [2,3,4]
print(sample_indices)
test_data = torch.utils.data.Subset(cifar10_test, sample_indices)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

[2, 3, 4]


In [ ]:


# Get a sample of 100 test examples
sample_indices = torch.randperm(len(cifar10_test))[:1]
test_data = torch.utils.data.Subset(cifar10_test, sample_indices)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
accuracy, f1, precision, recall, auroc = poisone_sisa.evaluate(test_loader, poisone_sisa.aggregate_majority_vote)

print(f'Accuracy: {accuracy}, F1-score: {f1}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

In [ ]:
accuracy, f1, precision, recall, auroc = poisone_sisa.evaluate(test_loader, poisone_sisa.aggregate_majority_vote)

print(f'Accuracy: {accuracy}, F1-score: {f1}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 8, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 3, 6, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 0, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 0, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 0, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 0, 8, 7, 1, 3, 0, 5, 7, 9, 3, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 6, 4, 6, 6, 5, 1, 3, 8, 8, 0, 4, 0, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 5, 0, 2, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 9, 2, 1, 6, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 5, 1, 7, 5, 4, 6

In [ ]:
# Step 5: Calculate the Attack Success Rate (ASR)
def calculate_asr(sisa, dataset, num_samples, target_class=0):
    # Prepare poisoned images from other classes
    poisoned_datasets = []

    for i in range(1, 10):
        if i != target_class:
            poisoned_indices = select_and_poison_data_inplace(dataset, target_class=i, num_samples=num_samples, block_size=3)
            poisoned_subset = Subset(dataset, poisoned_indices)
            poisoned_datasets.append(poisoned_subset)


    # Concatenate all poisoned subsets into one dataset
    poisoned_dataset = ConcatDataset(poisoned_datasets)
    poisoned_loader = DataLoader(poisoned_dataset, batch_size=64, shuffle=False)

    aggregated_predictions, _ = sisa.aggregate_simple_average(poisoned_loader)
    #accuracy, f1, precision, recall, auroc = poisone_sisa.evaluate(poisoned_loader, poisone_sisa.aggregate_simple_average)



    print(aggregated_predictions)

    misclassified_count = (aggregated_predictions == target_class).sum().item()
    total_count = len(poisoned_dataset)

    return misclassified_count / total_count

In [ ]:
asr = calculate_asr(poisone_sisa, cifar10_test, 100, 0)
print(f'Attack Success Rate (ASR): {asr * 100:.2f}%')


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,

# **Add On. Simulation Question 2.**
 Unlearn the same 500 data, and evaluate the model’s
performance, using all the same metrics ass before, on clean test data to ensure general performance is not significantly degraded. Also, calculate the attack success rate (ASR) as the
percentage of test samples misclassified as the target class.

In [ ]:
poisone_sisa.unlearn_with_indices(cifar10_train, poisoned_indices)

[42044, 3880, 33447, 8392, 36106, 31756, 29884, 22665, 34318, 20985, 3515, 45156, 43775, 30172, 1434, 43454, 555, 48032, 9362, 44599, 20790, 23808, 6867, 48908, 14592, 18951, 48856, 39083, 34998, 371, 8748, 35162, 21777, 21863, 40807, 15580, 13238, 10867, 43525, 47336, 26723, 9714, 6393, 47931, 23056, 4270, 49344, 13181, 47994, 17884, 18630, 29513, 33386, 37962, 30914, 29641, 18599, 41613, 41277, 1340, 1593, 41731, 49236, 48851, 30595, 25789, 36886, 47207, 36155, 47375, 16825, 30959, 34313, 48470, 8086, 11078, 21758, 39650, 27850, 22418, 42952, 29497, 12409, 902, 11100, 15755, 17620, 3076, 2932, 14393, 22026, 5477, 9384, 17133, 19814, 29854, 2718, 23096, 4798, 7674, 7889, 20523, 35626, 12192, 11709, 42688, 38519, 37978, 16838, 5703, 38224, 42231, 1711, 47379, 10578, 1188, 42998, 29341, 35411, 27931, 28449, 44299, 24096, 6563, 6276, 36606, 22997, 2391, 7251, 8313, 29880, 18315, 46706, 46200, 34547, 41975, 36819, 48119, 8116, 30257, 39664, 5623, 19011, 44946, 8515, 2066, 25070, 16238, 29

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 0.9033203125
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 0.5717366933822632
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 0.13786204159259796
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.09931378811597824
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.08478694409132004
0  --->  4000
overlap:  {3076, 902, 2950, 3335, 3466, 2189, 2066, 2453, 1434, 2718, 2848, 3362, 1188, 3880, 555, 557, 1709, 1711, 3246, 439, 1335, 2233, 1593, 3515, 1340, 1594, 1980, 965, 199, 332, 2391, 481, 3689, 2287, 3184, 497, 371, 2932, 115, 3066}
difference:  {3076, 2950, 902, 3335, 3466, 2189, 2066, 2453, 1434, 2718, 2848, 3362, 1188, 3880, 555, 557, 1709, 1711, 3246, 439, 1335, 2233, 1593, 3515, 1340, 1594, 1980, 965, 199, 332, 2391, 481, 3689, 2287, 3184, 497, 371, 2932, 115, 3066}
idx_slice != 1
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.6657801270484924
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.25337472558021545
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.04309507831931114
@

In [ ]:
# Step 4: Evaluate the model's performance on clean test data
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [ ]:
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

In [ ]:
accuracy, f1, precision, recall, auroc = poisone_sisa.evaluate(test_loader, poisone_sisa.aggregate_majority_vote)

print(f'Accuracy: {accuracy}, F1-score: {f1}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 8, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 3, 6, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 0, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 0, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 0, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 0, 8, 7, 1, 3, 0, 5, 7, 9, 3, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 6, 4, 6, 6, 5, 1, 3, 8, 8, 0, 4, 0, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 5, 0, 2, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 9, 2, 1, 6, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 5, 1, 7, 5, 4, 6

In [ ]:
poisone_sisa.load_models()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


In [ ]:
# Step 5: Calculate the Attack Success Rate (ASR)

asr_unlearn = calculate_asr(poisone_sisa, cifar10_test, 100, 0)
print(f'Attack Success Rate (ASR): {asr_unlearn * 100:.2f}%')


tensor([9, 3, 9, 0, 1, 9, 9, 9, 9, 3, 0, 9, 9, 9, 9, 9, 9, 1, 3, 9, 9, 8, 9, 9,
        1, 9, 9, 9, 9, 3, 9, 9, 0, 9, 9, 9, 9, 3, 9, 9, 3, 9, 9, 9, 3, 9, 8, 9,
        3, 9, 1, 9, 3, 0, 3, 9, 9, 3, 9, 9, 9, 9, 9, 1, 9, 9, 1, 9, 3, 9, 9, 3,
        9, 3, 9, 9, 1, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 0, 9, 3, 9, 9, 3, 9,
        2, 9, 3, 3, 3, 0, 3, 3, 6, 8, 3, 3, 2, 2, 3, 8, 9, 2, 2, 3, 9, 3, 2, 3,
        8, 0, 8, 8, 3, 3, 3, 2, 3, 3, 2, 3, 2, 3, 3, 0, 2, 2, 3, 0, 3, 3, 2, 8,
        3, 3, 2, 3, 2, 3, 0, 8, 2, 3, 3, 2, 3, 2, 0, 3, 2, 3, 3, 0, 3, 2, 3, 3,
        2, 0, 2, 3, 0, 0, 3, 3, 3, 0, 3, 9, 2, 3, 2, 3, 0, 3, 3, 0, 3, 9, 0, 2,
        8, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2, 8, 2, 3, 3, 3, 3, 3, 3, 3, 8, 9,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 0, 3, 2, 0,
        3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 2, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 0, 3, 4, 3, 3, 3, 3, 9,

In [ ]:
# Step 5: Calculate the Attack Success Rate (ASR)

asr_unlearn = calculate_asr(poisone_sisa, cifar10_test, 100, 0)
print(f'Attack Success Rate (ASR): {asr_unlearn * 100:.2f}%')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8,